In [1]:
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df = pd.read_csv('./Data/Final_cleaned_data.csv', lineterminator='\n')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,brand,model,rating,review,review_date,review_location,reviewer_name,trial_nights,warranty_years
0,purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,2021-02-22,United States,Jessica G.,100.0,10.0
1,purple,The Purple Mattress,5.0,First off I am a disabled veteran with insomni...,2017-09-19,"Warrenton, Virginia",Jared B.,100.0,10.0
2,purple,The Purple Mattress,1.0,This mattress is well made and the process of ...,2021-03-05,United States,LeAnn M.,100.0,10.0
3,purple,The Purple Mattress,3.0,The regular twin foam mattress was marked as a...,2021-03-04,United States,Kathryn W.,100.0,10.0
4,purple,The Purple Mattress,4.0,We had a premium mattress for years (Sterns an...,2021-03-04,United States,Adam R.,100.0,10.0


In [4]:
df.loc[(df['brand'] == 'Sealy') & (df['model'] == 'Chill Memory Foam Mattress'), 'model'] = 'Cocoon Chill'

In [5]:
import psycopg2
conn = psycopg2.connect('dbname=Mattresses')
cur = conn.cursor()
sql = 'SELECT * FROM mattressinfo'

In [7]:
mattress_info_df = pd.read_sql_query(sql, conn)
mattress_info_df

,id,brand,model,bedsize,bedtype,price
0,7,Tempur Pedic,TEMPUR-Adapt,Twin,Hybrid,1699.0
1,8,Tempur Pedic,TEMPUR-Adapt,Twin Long,Hybrid,1699.0
2,9,Tempur Pedic,TEMPUR-Adapt,Full,Hybrid,2049.0
3,10,Tempur Pedic,TEMPUR-Adapt,Queen,Hybrid,2199.0
4,11,Tempur Pedic,TEMPUR-Adapt,King,Hybrid,2899.0
...,...,...,...,...,...,...
143,80,Leesa,Leesa Hybrid Mattress,Twin Long,Hybrid,1049.0
144,81,Leesa,Leesa Hybrid Mattress,Full,Hybrid,1299.0
145,82,Leesa,Leesa Hybrid Mattress,Queen,Hybrid,1549.0
146,83,Leesa,Leesa Hybrid Mattress,King,Hybrid,1699.0


In [6]:
mattress_info_df.to_csv('mattress_info.csv')

In [7]:
df['brand'] = df['brand'].apply(lambda x: x.title())

In [8]:
df = df.merge(mattress_info_df, on=['brand', 'model'], how='left')

In [9]:
df

,brand,model,rating,review,review_date,review_location,reviewer_name,trial_nights,warranty_years,id,bedsize,bedtype,price
0,Purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,2021-02-22,United States,Jessica G.,100.0,10.0,27,Twin,Foam,599.0
1,Purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,2021-02-22,United States,Jessica G.,100.0,10.0,28,Twin Long,Foam,799.0
2,Purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,2021-02-22,United States,Jessica G.,100.0,10.0,29,Full,Foam,999.0
3,Purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,2021-02-22,United States,Jessica G.,100.0,10.0,30,Queen,Foam,1149.0
4,Purple,The Purple Mattress,5.0,I am a hard worker with way more to physically...,2021-02-22,United States,Jessica G.,100.0,10.0,31,King,Foam,1449.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143007,Casper,Element Mattress,5.0,"The Essential is a great mattress, especially ...",2018-07-24,"New York, NY",Lucia,100.0,10.0,56,Twin Long,Foam,445.0
143008,Casper,Element Mattress,5.0,"The Essential is a great mattress, especially ...",2018-07-24,"New York, NY",Lucia,100.0,10.0,57,Full,Foam,545.0
143009,Casper,Element Mattress,5.0,"The Essential is a great mattress, especially ...",2018-07-24,"New York, NY",Lucia,100.0,10.0,58,Queen,Foam,595.0
143010,Casper,Element Mattress,5.0,"The Essential is a great mattress, especially ...",2018-07-24,"New York, NY",Lucia,100.0,10.0,59,King,Foam,795.0


In [10]:
df.isna().sum()

brand                 0
model                 0
rating                0
review                0
review_date           0
review_location    5658
reviewer_name      4206
trial_nights          0
warranty_years        0
id                    0
bedsize               0
bedtype               0
price                 0
dtype: int64

In [11]:
df['key_words'] = ''

In [12]:
r = Rake()
for index, row in df.iterrows():
    r.extract_keywords_from_text(row['review'])
    key_words_dict_score = r.get_word_degrees()
    df.loc[index, 'key_words'] = ' '.join(list(key_words_dict_score.keys()))

In [13]:
for brand in df.brand.unique():
    df.loc[df['brand'] == brand, 'key_words'] += ' ' + str(brand)

In [14]:
df['key_words'] = df.apply(lambda row: row['key_words'] + ' {} star rating {} size bed {} dollars'.format(
row['rating'], row['bedsize'], row['price']), axis=1)

In [ ]:
df.to_csv('df_w_key_words.csv')

In [15]:
df.key_words[1]

'longer bolt less overall back cannot lay use purple mattress seems feeling rested curvy body allows 3 shorten turn truly handle day noticed remarkable differences broad shoulders three different places bed every night instead ranching toss simple overexertion days physically live working issues tension therapist going morning comfortably another minute go contributes received work healing time exponentially happened get relief foam roller matter sure lifetime 30 bent hurt life way big hips 4 00 rare woken support changed narrow waist actually looked forward sink something hard worker Purple 5.0 star rating Twin Long size bed 799.0 dollars'

In [ ]:
tfidf = TfidfVectorizer()

tfidf_matrix = tfidf.fit_transform(df['key_words'])

cosine_sim = pd.DataFrame(cosine_similarity(tfidf_matrix, tfidf_matrix))
cosine_sim

In [ ]:
test = 'I am a side sleeper. I want a Queen size bed, that is around 2000 dollars.'

In [ ]:
r.extract_keywords_from_text(test)
test_key_words_dict_score = r.get_word_degrees()
test = [' '.join(list(test_key_words_dict_score.keys()))]
text_matrix = tfidf.transform(cleaned_text)

In [ ]:
results = pd.DataFrame(cosine_similarity(tfidf_matrix, test)).sort_values(0, ascending=False).head(10)

In [ ]:
mattress_result_count = df.copy()
mattress_result_count = mattress_result_count[0:0]

for index in results.index:
    mattress_result_count = mattress_result_count.append(df.loc[index])

top_mattress = mattress_result_count.groupby(['brand', 'price', 'size']).count()['review']\
                .reset_index().sort_values(['review', 'size'], ascending=False).iloc[0]

In [ ]:
print('The best Mattress for you is a: \n')
print('{} {} mattress, which is ${}'.format(top_mattress['brand'].title(), top_mattress['size'], 
                                            int(top_mattress['price'])))